In [136]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [137]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "MAX-STD"

In [138]:
# /datassd/WHOLEIMAGE_MAMIP/EXTRACTED_FEATURES
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [139]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.519982,0.479010,0.532104,0.329144,0.713297,0.522315,0.614356,0.551377,0.833706,...,0.274570,0.519983,0.693129,0.425480,0.881372,0.541983,0.644103,0.661937,0.527496,0.821651
1,CHUM-002,0.331772,0.534072,1.220490,0.429507,0.632341,0.505171,0.562052,0.390735,0.266213,...,0.517452,0.612039,0.544685,0.853127,0.577021,0.526759,0.704940,0.371904,0.718858,1.322492
2,CHUM-006,0.501219,0.311402,0.853474,0.396596,0.654741,0.658689,0.731430,0.487003,0.479731,...,0.169724,0.696628,0.377406,0.278703,0.547150,0.553275,0.742782,0.324220,0.519851,1.574138
3,CHUM-007,0.480090,0.647200,0.538190,0.232534,0.492256,0.439751,0.690010,0.261845,0.422919,...,0.170912,0.399160,0.434155,0.321014,0.676776,0.474006,0.687809,0.142707,0.316534,1.584920
4,CHUM-008,0.333042,0.279040,0.511778,0.389198,0.879973,0.447316,0.680153,0.305762,0.383276,...,0.435639,0.937325,0.945857,0.196866,1.028598,0.351773,0.840229,0.427045,0.557202,0.679726


# Up to here

Read Outcome csv

In [140]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [141]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [142]:
OD.columns[0]

'Patient_ID'

In [143]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [144]:
OD.columns[0]


'Patient_ID'

In [145]:
PET_outcome_data.columns[0]

'Patient_ID'

In [146]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.519982,0.479010,0.532104,0.329144,0.713297,0.522315,0.614356,0.551377,0.833706,...,0.274570,0.519983,0.693129,0.425480,0.881372,0.541983,0.644103,0.661937,0.527496,0.821651
1,CHUM-002,0.331772,0.534072,1.220490,0.429507,0.632341,0.505171,0.562052,0.390735,0.266213,...,0.517452,0.612039,0.544685,0.853127,0.577021,0.526759,0.704940,0.371904,0.718858,1.322492
2,CHUM-006,0.501219,0.311402,0.853474,0.396596,0.654741,0.658689,0.731430,0.487003,0.479731,...,0.169724,0.696628,0.377406,0.278703,0.547150,0.553275,0.742782,0.324220,0.519851,1.574138
3,CHUM-007,0.480090,0.647200,0.538190,0.232534,0.492256,0.439751,0.690010,0.261845,0.422919,...,0.170912,0.399160,0.434155,0.321014,0.676776,0.474006,0.687809,0.142707,0.316534,1.584920
4,CHUM-008,0.333042,0.279040,0.511778,0.389198,0.879973,0.447316,0.680153,0.305762,0.383276,...,0.435639,0.937325,0.945857,0.196866,1.028598,0.351773,0.840229,0.427045,0.557202,0.679726


In [147]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [148]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [149]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,0.519982,0.479010,0.532104,0.329144,0.713297,0.522315,0.614356,0.551377,0.833706,...,0.693129,0.425480,0.881372,0.541983,0.644103,0.661937,0.527496,0.821651,0,1704
1,CHUM-002,0.331772,0.534072,1.220490,0.429507,0.632341,0.505171,0.562052,0.390735,0.266213,...,0.544685,0.853127,0.577021,0.526759,0.704940,0.371904,0.718858,1.322492,1,439
2,CHUM-006,0.501219,0.311402,0.853474,0.396596,0.654741,0.658689,0.731430,0.487003,0.479731,...,0.377406,0.278703,0.547150,0.553275,0.742782,0.324220,0.519851,1.574138,0,1186
3,CHUM-007,0.480090,0.647200,0.538190,0.232534,0.492256,0.439751,0.690010,0.261845,0.422919,...,0.434155,0.321014,0.676776,0.474006,0.687809,0.142707,0.316534,1.584920,0,1702
4,CHUM-008,0.333042,0.279040,0.511778,0.389198,0.879973,0.447316,0.680153,0.305762,0.383276,...,0.945857,0.196866,1.028598,0.351773,0.840229,0.427045,0.557202,0.679726,0,1499


In [150]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)